In [ ]:
#Load all csv
import pandas as pd
import numpy as np

number_of_testers = 34
filenames = ["SL_LIT", "SL_BIG", "FA_LIT", "FA_BIG"]
animname = ["VB", "HS"]
all_dfs = {}

for tester in range(1, number_of_testers+1):
    for session in range(1, 4):
        for trial in range(1, 4):
            for fname in filenames:
                for anim_name in animname:
                    key = f"T{tester}_S{session}_TRY{trial}_{anim_name}_{fname}"
                    path = rf"C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Results_csv\Tester{tester}\Session{session}\Trial{trial}\T{tester}-S{session}-TRY{trial}-{anim_name}_{fname}.csv"
                    try:
                        df = pd.read_csv(path)
                        all_dfs[key] = df
                        print(f"Loaded: {key}")
                    except FileNotFoundError:
                        print(f"Missing: {key}")


Loaded: T1_S1_TRY1_VB_SL_LIT
Loaded: T1_S1_TRY1_HS_SL_LIT
Loaded: T1_S1_TRY1_VB_SL_BIG
Loaded: T1_S1_TRY1_HS_SL_BIG
Loaded: T1_S1_TRY1_VB_FA_LIT
Loaded: T1_S1_TRY1_HS_FA_LIT
Loaded: T1_S1_TRY1_VB_FA_BIG
Loaded: T1_S1_TRY1_HS_FA_BIG
Loaded: T1_S1_TRY2_VB_SL_LIT
Loaded: T1_S1_TRY2_HS_SL_LIT
Loaded: T1_S1_TRY2_VB_SL_BIG
Loaded: T1_S1_TRY2_HS_SL_BIG
Loaded: T1_S1_TRY2_VB_FA_LIT
Loaded: T1_S1_TRY2_HS_FA_LIT
Loaded: T1_S1_TRY2_VB_FA_BIG
Loaded: T1_S1_TRY2_HS_FA_BIG
Loaded: T1_S1_TRY3_VB_SL_LIT
Loaded: T1_S1_TRY3_HS_SL_LIT
Loaded: T1_S1_TRY3_VB_SL_BIG
Loaded: T1_S1_TRY3_HS_SL_BIG
Loaded: T1_S1_TRY3_VB_FA_LIT
Loaded: T1_S1_TRY3_HS_FA_LIT
Loaded: T1_S1_TRY3_VB_FA_BIG
Loaded: T1_S1_TRY3_HS_FA_BIG
Loaded: T1_S2_TRY1_VB_SL_LIT
Loaded: T1_S2_TRY1_HS_SL_LIT
Loaded: T1_S2_TRY1_VB_SL_BIG
Loaded: T1_S2_TRY1_HS_SL_BIG
Loaded: T1_S2_TRY1_VB_FA_LIT
Loaded: T1_S2_TRY1_HS_FA_LIT
Loaded: T1_S2_TRY1_VB_FA_BIG
Loaded: T1_S2_TRY1_HS_FA_BIG
Loaded: T1_S2_TRY2_VB_SL_LIT
Loaded: T1_S2_TRY2_HS_SL_LIT
Loaded: T1_S2_

In [ ]:
#Fixation vector for each file
import duckdb

fpogs_vector = []

# Loop through all DataFrames in the dictionary
for key, df in all_dfs.items():
    
    result = duckdb.query("""
        SELECT 
            AVG(FPOGX) AS FPOGX, 
            AVG(FPOGY) AS FPOGY, 
            MAX(FPOGS) AS FPOGS, 
            MAX(FPOGD) AS FPOGD, 
            FPOGID
        FROM df
        WHERE FPOGV = '1'
        GROUP BY FPOGID
        ORDER BY FPOGID
    """).to_df()
    
    # Extract vectors from result
    fpogx_vector = result['FPOGX'].tolist()
    fpogy_vector = result['FPOGY'].tolist()
    fpogs_values = result['FPOGS'].tolist()
    fpogd_vector = result['FPOGD'].tolist()
    fpogid_vector = result['FPOGID'].tolist()
    
    # Append structured result
    fpogs_vector.append({
        'file_key': key,
        'FPOGX': fpogx_vector,
        'FPOGY': fpogy_vector,
        'FPOGS': fpogs_values,
        'FPOGD': fpogd_vector,
        'FPOGID': fpogid_vector
    })

# Convert to DataFrame
fpogs_vector_df = pd.DataFrame(fpogs_vector)

# Display the DataFrame
display(fpogs_vector_df)


In [ ]:
#Show the fixation data in a nicer way into a csv file
import pandas as pd

# Flatten the list into a structured tabular format
flattened_fpogs_data = []

for item in fpogs_vector:
    file_key = item['file_key']
    for fpogid, fpogx, fpogy, fpogs, fpogd in zip(
        item['FPOGID'], item['FPOGX'], item['FPOGY'], item['FPOGS'], item['FPOGD']
    ):
        flattened_fpogs_data.append({
            'file_key': file_key,
            'FPOGID': fpogid,
            'FPOGX': fpogx,
            'FPOGY': fpogy,
            'FPOGS': fpogs,
            'FPOGD': fpogd
        })

# Create the final DataFrame
fpogs_table_df = pd.DataFrame(flattened_fpogs_data)

# Display the table
display(fpogs_table_df)

#Create the csv 
fpogs_table_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\fixation_vector.csv', index=False)


In [ ]:
#Create the dataframe with the first column, which is the number of fixations
feature_vectors = []
# Loop through all files in the dataframe
for key, df in all_dfs.items():
    #Start from 1 because the fixation with id zero is our first fixation
    num_fix = 1
    if 'FPOGID' in df.columns and not df.empty:
        # Convert to list for easier iteration
        fix_id_series = df['FPOGID'].tolist() 
        # Check that the list is not empty
        if fix_id_series: 
            previous_id = fix_id_series[0]
        # Start from the second element
        for current_id in fix_id_series[1:]:
            if current_id != previous_id:
                num_fix += 1
                previous_id = current_id

    # Create a feature vector with key and features
    feature_vectors.append({
        'file_key': key,
        'f0': num_fix
    })
     
# Create a DataFrame from all feature vectors
feature_df = pd.DataFrame(feature_vectors)

# Display the current dataset
display(feature_df)


In [ ]:
#Obtain the duration of every FPOGID in a file
import duckdb

fix_duration_vector = []

for fix in fpogs_vector:
    
    fpogd_vector = fix['FPOGD']
    fpogid_vector = fix['FPOGID']

    # Store in the feature vector list
    fix_duration_vector.append({
        'file_key': key,
        'FPOGID': fpogid_vector,
        'FPOGD': fpogd_vector
    })

# Create a final DataFrame
fix_duration_vector_df = pd.DataFrame(fix_duration_vector)

# Display the DataFrame
display(fix_duration_vector_df)


In [ ]:
#Minimum fixation duration

min_fix_data = []

for item in fpogs_vector:
    file_key = item['file_key']
    fpogd_list = item['FPOGD']
    
    if fpogd_list:
        min_fpogd = np.min(fpogd_list)
    else:
        min_fpogd = None  
    
    min_fix_data.append({ 
        'file_key': file_key,
        'f1': min_fpogd
    })

# Convert to DataFrame for easy use or merging
min_fixation_df = pd.DataFrame(min_fix_data)

feature_df = feature_df.merge(min_fixation_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

In [ ]:
#Maximum fixation duration that join with the feature vector
max_fixation_data = []

for item in fpogs_vector:
    file_key = item['file_key']
    max_fpogd = item['FPOGD']

    #Check if empty
    if max_fpogd: 
        max_duration = max(max_fpogd)
    else:
        max_duration = None 

    max_fixation_data.append({
        'file_key': file_key,
        'f2': max_duration
    })

# Create pandas DataFrame
max_fixation_df = pd.DataFrame(max_fixation_data)

feature_df = feature_df.merge(max_fixation_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

In [ ]:
#Arithmetic mean of fixation duration that join with feature vector
mean_fixation_data = []

for item in fpogs_vector:
    file_key = item['file_key']
    max_fpogd = item['FPOGD']

    #Check if empty
    if max_fpogd: 
        mean_duration = np.mean(max_fpogd)
    else:
        mean_duration = None 

    mean_fixation_data.append({
        'file_key': file_key,
        'f3': mean_duration
    })

# Create pandas DataFrame
mean_fixation_df = pd.DataFrame(mean_fixation_data)

feature_df = feature_df.merge(mean_fixation_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


In [ ]:
#Geometric mean of fixation duration that join with feature vector
from scipy.stats import gmean

geom_mean_fixation_data = []

for item in fpogs_vector:
    file_key = item['file_key']
    max_fpogd = item['FPOGD']

    #Check if empty
    if max_fpogd: 
        geom_mean_duration = gmean(max_fpogd)
    else:
        geom_mean_duration = None 

    geom_mean_fixation_data.append({
        'file_key': file_key,
        'f4': geom_mean_duration
    })

# Create pandas DataFrame
geom_mean_fixation_df = pd.DataFrame(geom_mean_fixation_data)

feature_df = feature_df.merge(geom_mean_fixation_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


In [ ]:
#Median of fixation duration that join with feature vector
median_fixation_data = []

for item in fpogs_vector:
    file_key = item['file_key']
    max_fpogd = item['FPOGD']

    #Check if empty
    if max_fpogd: 
        median_duration = np.median(max_fpogd)
    else:
        median_duration = None 

    median_fixation_data.append({
        'file_key': file_key,
        'f5': median_duration
    })

# Create pandas DataFrame
median_fixation_df = pd.DataFrame(median_fixation_data)

feature_df = feature_df.merge(median_fixation_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


In [ ]:
#Standard deviation of fixation duration that join with feature vector
std_fixation_data = []

for item in fpogs_vector:
    file_key = item['file_key']
    max_fpogd = item['FPOGD']

    #Check if empty
    if max_fpogd: 
        std_duration = np.std(max_fpogd)
    else:
        std_duration = None 

    std_fixation_data.append({
        'file_key': file_key,
        'f6': std_duration
    })

# Create pandas DataFrame
std_fixation_df = pd.DataFrame(std_fixation_data)

feature_df = feature_df.merge(std_fixation_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


In [ ]:
#Median absolute deviation of fixation duration that join with feature vector
from scipy.stats import median_abs_deviation

mad_fixation_data = []

for item in fpogs_vector:
    file_key = item['file_key']
    max_fpogd = item['FPOGD']

    #Check if empty
    if max_fpogd: 
        mad_duration = median_abs_deviation(max_fpogd, scale=1)
    else:
        mad_duration = None 

    mad_fixation_data.append({
        'file_key': file_key,
        'f7': mad_duration
    })

# Create pandas DataFrame
mad_fixation_df = pd.DataFrame(mad_fixation_data)

feature_df = feature_df.merge(mad_fixation_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


In [ ]:
#Skewness of fixation duration that join with feature vector
from scipy.stats import skew

skew_fixation_data = []

for item in fpogs_vector:
    file_key = item['file_key']
    max_fpogd = item['FPOGD']

    #Check if empty
    if max_fpogd: 
        skew_duration = skew(max_fpogd)
    else:
        skew_duration = None 

    skew_fixation_data.append({
        'file_key': file_key,
        'f8': skew_duration
    })

# Create pandas DataFrame
skew_fixation_df = pd.DataFrame(skew_fixation_data)

feature_df = feature_df.merge(skew_fixation_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


In [ ]:
#Interquartile range of fixation duration that join with feature vector
from scipy.stats import iqr

iqr_fixation_data = []

for item in fpogs_vector:
    file_key = item['file_key']
    max_fpogd = item['FPOGD']

    #Check if empty
    if max_fpogd: 
        iqr_duration = iqr(max_fpogd)
    else:
        iqr_duration = None 

    iqr_fixation_data.append({
        'file_key': file_key,
        'f9': iqr_duration
    })

# Create pandas DataFrame
iqr_fixation_df = pd.DataFrame(iqr_fixation_data)

feature_df = feature_df.merge(iqr_fixation_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


In [ ]:
#Kurtosis of fixation duration that join with feature vector
from scipy.stats import kurtosis

kurt_fixation_data = []

for item in fpogs_vector:
    file_key = item['file_key']
    max_fpogd = item['FPOGD']

    #Check if empty
    if max_fpogd: 
        kurt_duration = kurtosis(max_fpogd)
    else:
        kurt_duration = None 

    kurt_fixation_data.append({
        'file_key': file_key,
        'f10': kurt_duration
    })

# Create pandas DataFrame
kurt_fixation_df = pd.DataFrame(kurt_fixation_data)

feature_df = feature_df.merge(kurt_fixation_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


In [ ]:
#Distance between consecutive fixations
#The distance is computed taking in consideration the euclidean distance from the average FPOGX and FPOGY of two consecutive fixations
from scipy.spatial import distance

fpogs_diff_vector = []

for fix in fpogs_vector:
    file_key = fix['file_key']
    fpogid_vector = fix['FPOGID']
    fpogx_vector = fix['FPOGX']
    fpogy_vector = fix['FPOGY']

    # Build list of consecutive FPOGID pairs 
    fpogid_pairs = []
    for i in range(len(fpogid_vector) - 1):
        pair = (fpogid_vector[i], fpogid_vector[i + 1])
        fpogid_pairs.append(pair)

    # Build list of consecutive Euclidean distances
    fpogs_differences = []
    for i in range(len(fpogx_vector) - 1):
            point1 = [fpogx_vector[i], fpogy_vector[i]]
            point2 = [fpogx_vector[i + 1], fpogy_vector[i + 1]]
            dist = distance.euclidean(point1, point2)
            fpogs_differences.append(dist)
       
    # Append result 
    fpogs_diff_vector.append({
        'file_key': file_key,
        'FPOGID_pairs': fpogid_pairs,
        'FPOGS_differences': fpogs_differences
    })

# Create the DataFrame
fpogs_diff_vector_df = pd.DataFrame(fpogs_diff_vector)

# Display the DataFrame
display(fpogs_diff_vector_df)


In [ ]:
#Minimum difference between consecutive fixations that join with feature vector
min_diff_data = []

for item in fpogs_diff_vector:
    file_key = item['file_key']
    differences = item['FPOGS_differences']

    #Check if the list is empty
    if differences:  
        min_diff = np.min(differences)
    else:
        min_diff = None

    min_diff_data.append({
        'file_key': file_key,
        'f11': min_diff
    })

# Create DataFrame from min differences
min_diff_df = pd.DataFrame(min_diff_data)

feature_df = feature_df.merge(min_diff_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

In [ ]:
#Maximum difference between consecutive fixations that join with feature vector
max_diff_data = []

for item in fpogs_diff_vector:
    file_key = item['file_key']
    differences = item['FPOGS_differences']

    #Check if the list is empty
    if differences:  
        max_diff = np.max(differences)
    else:
        max_diff = None

    max_diff_data.append({
        'file_key': file_key,
        'f12': max_diff
    })

# Create the DataFrame 
max_diff_df = pd.DataFrame(max_diff_data)

feature_df = feature_df.merge(max_diff_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

In [ ]:
#Arithmetic mean between consecutive fixations that join with feature vector
mean_diff_data = []

for item in fpogs_diff_vector:
    file_key = item['file_key']
    differences = item['FPOGS_differences']

    #Check if the list is empty
    if differences:  
        mean_diff = np.mean(differences)
    else:
        mean_diff = None

    mean_diff_data.append({
        'file_key': file_key,
        'f13': mean_diff
    })

# Create the DataFrame 
mean_diff_df = pd.DataFrame(mean_diff_data)

feature_df = feature_df.merge(mean_diff_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

In [ ]:
#Geometric mean between consecutive fixations that join with feature vector
gmean_diff_data = []

for item in fpogs_diff_vector:
    file_key = item['file_key']
    differences = item['FPOGS_differences']

    #Check if the list is empty
    if differences:  
        gmean_diff = gmean(differences)
    else:
        gmean_diff = None

    gmean_diff_data.append({
        'file_key': file_key,
        'f14': gmean_diff
    })

# Create the DataFrame 
gmean_diff_df = pd.DataFrame(gmean_diff_data)

feature_df = feature_df.merge(gmean_diff_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

In [ ]:
#Median between consecutive fixations that join with feature vector
median_diff_data = []

for item in fpogs_diff_vector:
    file_key = item['file_key']
    differences = item['FPOGS_differences']

    #Check if the list is empty
    if differences:  
        median_diff = np.median(differences)
    else:
        median_diff = None

    median_diff_data.append({
        'file_key': file_key,
        'f15': median_diff
    })

# Create the DataFrame 
median_diff_df = pd.DataFrame(median_diff_data)

feature_df = feature_df.merge(median_diff_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

In [ ]:
#STD between consecutive fixations that join with feature vector
std_diff_data = []

for item in fpogs_diff_vector:
    file_key = item['file_key']
    differences = item['FPOGS_differences']

    #Check if the list is empty
    if differences:  
        std_diff = np.std(differences)
    else:
        std_diff = None

    std_diff_data.append({
        'file_key': file_key,
        'f16': std_diff
    })

# Create the DataFrame 
std_diff_df = pd.DataFrame(std_diff_data)

feature_df = feature_df.merge(std_diff_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

In [ ]:
#MAD between consecutive fixations that join with feature vector
mad_diff_data = []

for item in fpogs_diff_vector:
    file_key = item['file_key']
    differences = item['FPOGS_differences']

    #Check if the list is empty
    if differences:  
        mad_diff = median_abs_deviation(differences, scale=1)
    else:
        mad_diff = None

    mad_diff_data.append({
        'file_key': file_key,
        'f17': mad_diff
    })

# Create the DataFrame 
mad_diff_df = pd.DataFrame(mad_diff_data)

feature_df = feature_df.merge(mad_diff_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

In [ ]:
#Skewness between consecutive fixations that join with feature vector
skew_diff_data = []

for item in fpogs_diff_vector:
    file_key = item['file_key']
    differences = item['FPOGS_differences']

    #Check if the list is empty
    if differences:  
        skew_diff = skew(differences)
    else:
        skew_diff = None

    skew_diff_data.append({
        'file_key': file_key,
        'f18': skew_diff
    })

# Create the DataFrame 
skew_diff_df = pd.DataFrame(skew_diff_data)

feature_df = feature_df.merge(skew_diff_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

In [ ]:
#Interquartile range between consecutive fixations that join with feature vector
iqr_diff_data = []

for item in fpogs_diff_vector:
    file_key = item['file_key']
    differences = item['FPOGS_differences']

    #Check if the list is empty
    if differences:  
        iqr_diff = iqr(differences)
    else:
        iqr_diff = None

    iqr_diff_data.append({
        'file_key': file_key,
        'f19': iqr_diff
    })

# Create the DataFrame 
iqr_diff_df = pd.DataFrame(iqr_diff_data)

feature_df = feature_df.merge(iqr_diff_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

In [ ]:
#Kurtosis between consecutive fixations that join with feature vector
kurt_diff_data = []

for item in fpogs_diff_vector:
    file_key = item['file_key']
    differences = item['FPOGS_differences']

    #Check if the list is empty
    if differences:  
        kurt_diff = kurtosis(differences)
    else:
        kurt_diff = None

    kurt_diff_data.append({
        'file_key': file_key,
        'f20': kurt_diff
    })

# Create the DataFrame 
kurt_diff_df = pd.DataFrame(kurt_diff_data)

feature_df = feature_df.merge(kurt_diff_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

In [ ]:
#Saccade vector for each file
fpogs_saccade_vector = []

# Loop through all DataFrames in the dictionary
for key, df in all_dfs.items():
    
    result = duckdb.query("""
        SELECT 
            AVG(FPOGX) AS FPOGX, 
            AVG(FPOGY) AS FPOGY, 
            MAX(FPOGS) AS FPOGS, 
            MAX(FPOGD) AS FPOGD, 
            FPOGID
        FROM df
        WHERE FPOGV = '0'
        GROUP BY FPOGID
        ORDER BY FPOGID
    """).to_df()
    
    # Extract vectors from result
    fpogx_vector = result['FPOGX'].tolist()
    fpogy_vector = result['FPOGY'].tolist()
    fpogs_values = result['FPOGS'].tolist()
    fpogd_vector = result['FPOGD'].tolist()
    fpogid_vector = result['FPOGID'].tolist()
    
    # Append structured result
    fpogs_saccade_vector.append({
        'file_key': key,
        'FPOGX': fpogx_vector,
        'FPOGY': fpogy_vector,
        'FPOGS': fpogs_values,
        'FPOGD': fpogd_vector,
        'FPOGID': fpogid_vector
    })

# Convert to DataFrame
fpogs_saccade_vector_df = pd.DataFrame(fpogs_saccade_vector)

# Display the DataFrame
display(fpogs_saccade_vector_df)


In [ ]:
#Show the saccade data in a nicer way into a csv file

# Flatten the list into a structured tabular format
flattened_fpogs_data = []

for item in fpogs_saccade_vector:
    file_key = item['file_key']
    for fpogid, fpogx, fpogy, fpogs, fpogd in zip(
        item['FPOGID'], item['FPOGX'], item['FPOGY'], item['FPOGS'], item['FPOGD']
    ):
        flattened_fpogs_data.append({
            'file_key': file_key,
            'FPOGID': fpogid,
            'FPOGX': fpogx,
            'FPOGY': fpogy,
            'FPOGS': fpogs,
            'FPOGD': fpogd
        })

# Create the final DataFrame
fpogs_table_df = pd.DataFrame(flattened_fpogs_data)

# Display the table
display(fpogs_table_df)

#Create the csv 
fpogs_table_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\saccade_vector.csv', index=False)


In [ ]:
#Obtain the speed of every saccade in a file

saccade_vector = []

for fix in fpogs_saccade_vector:
    file_key = fix['file_key']
    fpogd_vector = fix['FPOGD']
    fpogs_vector = fix['FPOGS']
    fpogid_vector = fix['FPOGID']
    fpogx_vector = fix['FPOGX']
    fpogy_vector = fix['FPOGY']

    # Build list of saccade speeds
    saccade_speeds = []
    id_pairs = []

    for i in range(len(fpogs_vector) - 1):
        #Euclidean distance between two consecutive saccades
        point1 = [fpogx_vector[i], fpogy_vector[i]]
        point2 = [fpogx_vector[i + 1], fpogy_vector[i + 1]]
        dist = distance.euclidean(point1, point2)
        
        #Time of the saccade. It is obtained from FPOGS[i+1] - FPOGS[i] which include the time of fixation + saccade.
        #To obtain only the saccade time I subract from this difference FPOGD[i] which is the duration of the fixation
        diff = fpogs_vector[i + 1] - fpogs_vector[i]
        time = diff - fpogd_vector[i]

        #Compute the speed
        if time > 0:
            speed = dist / time
        else:
            speed = np.nan  

        saccade_speeds.append(speed)
        id_pairs.append((fpogid_vector[i], fpogid_vector[i + 1]))

    saccade_vector.append({
        'file_key': file_key,
        'FPOGID_pairs': id_pairs,
        'FPOGS_speeds': saccade_speeds
    })

saccade_vector_df = pd.DataFrame(saccade_vector)
display(saccade_vector_df)


In [ ]:
#Minimum saccade speed
min_speed_data = []

for item in saccade_vector:
    file_key = item['file_key']
    speeds = item['FPOGS_speeds']

    #Check if the list is empty
    if speeds:  
        min_speed = np.min(speeds)
    else:
        min_speed = None

    min_speed_data.append({
        'file_key': file_key,
        'f21': min_speed
    })

# Create DataFrame from min speeds
min_speed_df = pd.DataFrame(min_speed_data)

feature_df = feature_df.merge(min_speed_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

In [ ]:
#Maximim saccade speed
max_speed_data = []

for item in saccade_vector:
    file_key = item['file_key']
    speeds = item['FPOGS_speeds']

    #Check if the list is empty
    if speeds:  
        max_speed = np.max(speeds)
    else:
        max_speed = None

    max_speed_data.append({
        'file_key': file_key,
        'f22': max_speed
    })

# Create DataFrame from max speeds
max_speed_df = pd.DataFrame(max_speed_data)

feature_df = feature_df.merge(max_speed_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

In [ ]:
#Arithmetic mean saccade speed
mean_speed_data = []

for item in saccade_vector:
    file_key = item['file_key']
    speeds = item['FPOGS_speeds']

    #Check if the list is empty
    if speeds:  
        mean_speed = np.mean(speeds)
    else:
        mean_speed = None

    mean_speed_data.append({
        'file_key': file_key,
        'f23': mean_speed
    })

# Create DataFrame from mean speeds
mean_speed_df = pd.DataFrame(mean_speed_data)

feature_df = feature_df.merge(mean_speed_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

In [ ]:
#Geometric mean saccade speed
gmean_speed_data = []

for item in saccade_vector:
    file_key = item['file_key']
    speeds = item['FPOGS_speeds']

    #Check if the list is empty
    if speeds:  
        gmean_speed = gmean(speeds)
    else:
        gmean_speed = None

    gmean_speed_data.append({
        'file_key': file_key,
        'f24': gmean_speed
    })

# Create DataFrame from gmean speeds
gmean_speed_df = pd.DataFrame(gmean_speed_data)

feature_df = feature_df.merge(gmean_speed_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

In [ ]:
#Median saccade speed
median_speed_data = []

for item in saccade_vector:
    file_key = item['file_key']
    speeds = item['FPOGS_speeds']

    #Check if the list is empty
    if speeds:  
        median_speed = np.median(speeds)
    else:
        median_speed = None

    median_speed_data.append({
        'file_key': file_key,
        'f25': median_speed
    })

# Create DataFrame from median speeds
median_speed_df = pd.DataFrame(median_speed_data)

feature_df = feature_df.merge(median_speed_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

In [ ]:
#STD saccade speed
std_speed_data = []

for item in saccade_vector:
    file_key = item['file_key']
    speeds = item['FPOGS_speeds']

    #Check if the list is empty
    if speeds:  
        std_speed = np.std(speeds)
    else:
        std_speed = None

    std_speed_data.append({
        'file_key': file_key,
        'f26': std_speed
    })

# Create DataFrame from std speeds
std_speed_df = pd.DataFrame(std_speed_data)

feature_df = feature_df.merge(std_speed_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

In [ ]:
#MAD saccade speed
mad_speed_data = []

for item in saccade_vector:
    file_key = item['file_key']
    speeds = item['FPOGS_speeds']

    #Check if the list is empty
    if speeds:  
        mad_speed = median_abs_deviation(speeds, scale=1)
    else:
        mad_speed = None

    mad_speed_data.append({
        'file_key': file_key,
        'f27': mad_speed
    })

# Create DataFrame from mad speeds
mad_speed_df = pd.DataFrame(mad_speed_data)

feature_df = feature_df.merge(mad_speed_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

In [ ]:
#Skewness saccade speed
skew_speed_data = []

for item in saccade_vector:
    file_key = item['file_key']
    speeds = item['FPOGS_speeds']

    #Check if the list is empty
    if speeds:  
        skew_speed = skew(speeds)
    else:
        skew_speed = None

    skew_speed_data.append({
        'file_key': file_key,
        'f28': skew_speed
    })

# Create DataFrame from skew speeds
skew_speed_df = pd.DataFrame(skew_speed_data)

feature_df = feature_df.merge(skew_speed_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

In [ ]:
#IQR saccade speed
iqr_speed_data = []

for item in saccade_vector:
    file_key = item['file_key']
    speeds = item['FPOGS_speeds']

    #Check if the list is empty
    if speeds:  
        iqr_speed = iqr(speeds)
    else:
        iqr_speed = None

    iqr_speed_data.append({
        'file_key': file_key,
        'f29': iqr_speed
    })

# Create DataFrame from iqr speeds
iqr_speed_df = pd.DataFrame(iqr_speed_data)

feature_df = feature_df.merge(iqr_speed_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

In [ ]:
#Kurtosis saccade speed
kurt_speed_data = []

for item in saccade_vector:
    file_key = item['file_key']
    speeds = item['FPOGS_speeds']

    #Check if the list is empty
    if speeds:  
        kurt_speed = kurtosis(speeds)
    else:
        kurt_speed = None

    kurt_speed_data.append({
        'file_key': file_key,
        'f30': kurt_speed
    })

# Create DataFrame from kurt speeds
kurt_speed_df = pd.DataFrame(kurt_speed_data)

feature_df = feature_df.merge(kurt_speed_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

In [54]:
#Scanpath lenght for every file
#The scanpath lenght is computed summing the euclidean distance of every consecutive point in the file

fpogs_scan_vector = []

for fix in fpogs_vector:
    file_key = fix['file_key']
    fpogx_vector = fix['FPOGX']
    fpogy_vector = fix['FPOGY']

    tot = 0
    for i in range(len(fpogx_vector) - 1):
        point1 = [fpogx_vector[i], fpogy_vector[i]]
        point2 = [fpogx_vector[i + 1], fpogy_vector[i + 1]]
        dist = distance.euclidean(point1, point2)
        tot += dist

    fpogs_scan_vector.append({
        'file_key': file_key,
        'scanpath_length': tot
    })

fpogs_scan_vector_df = pd.DataFrame(fpogs_scan_vector)
display(fpogs_scan_vector_df)


TypeError: 'float' object is not subscriptable

In [25]:
#List of all valid(LPV = 1) left pupil diameter(LPD) for each file
lpd_vector = []

# Loop through all DataFrames in the dictionary
for key, df in all_dfs.items():
    # Query using DuckDB
    result = duckdb.query("""
        SELECT LPD 
        FROM df
        WHERE LPV = '1'
    """).to_df()

    # Extract vectors
    lpd_list = result['LPD'].tolist()

    # Append result 
    lpd_vector.append({
        'file_key': key,
        'LPD': lpd_list
    })

# Create the DataFrame
lpd_vector_df = pd.DataFrame(lpd_vector)

# Display the DataFrame
display(lpd_vector_df)


,file_key,LPD
0,T1_S1_TRY1_VB_SL_LIT,"[18.33092, 18.23724, 20.23254, 20.18498, 20.33..."
1,T1_S1_TRY1_HS_SL_LIT,"[20.73535, 20.51113, 20.60152, 20.485, 20.4977..."
2,T1_S1_TRY1_VB_SL_BIG,"[17.84583, 18.04667, 18.0429, 17.9603, 17.9667..."
3,T1_S1_TRY1_HS_SL_BIG,"[16.78424, 18.47666, 18.58866, 18.46712, 18.70..."
4,T1_S1_TRY1_VB_FA_LIT,"[20.1371, 20.03338, 19.02217, 19.03521, 19.026..."
...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,"[30.57337, 30.61803, 30.24858, 30.2585, 30.285..."
2324,T33_S3_TRY3_VB_FA_LIT,"[31.52806, 30.68255, 30.68035, 30.68035, 30.76..."
2325,T33_S3_TRY3_HS_FA_LIT,"[28.62523, 28.5755, 31.57859, 31.64881, 31.629..."
2326,T33_S3_TRY3_VB_FA_BIG,"[27.92934, 27.92934, 27.90902, 27.9475, 28.061..."


In [26]:
#Minimum left pupil diameter
min_lpd_data = []

for item in lpd_vector:
    file_key = item['file_key']
    lpd = item['LPD']

    #Check if the list is empty
    if lpd:  
        min_lpd = np.min(lpd)
    else:
        min_lpd = None

    min_lpd_data.append({
        'file_key': file_key,
        'f32': min_lpd
    })

# Create DataFrame from min differences
min_lpd_df = pd.DataFrame(min_lpd_data)

feature_df = feature_df.merge(min_lpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f12,f13,f14,f15,f16,f17,f18,f19,f20,f32
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959,...,2.79346,0.317581,0.264066,0.257935,0.350696,0.076720,6.272821,0.150022,41.389720,14.97845
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898,...,0.77747,0.349831,0.308199,0.294860,0.179607,0.114075,0.867020,0.266113,-0.160493,9.10561
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440,...,0.98633,0.265482,0.242950,0.244080,0.138565,0.044005,3.176352,0.086180,12.437481,14.20694
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425,...,2.69275,0.201254,0.156554,0.140630,0.287425,0.040160,7.426541,0.081420,60.650928,16.78424
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911,...,1.86181,0.281757,0.238197,0.241030,0.252794,0.063660,4.762304,0.123387,24.804236,16.44751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665,...,1.00458,0.396504,0.342790,0.311770,0.233169,0.097170,1.325127,0.241760,0.819504,25.92760
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215,...,0.89111,0.287688,0.261247,0.261295,0.138470,0.060455,1.884942,0.116243,5.161301,24.34467
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985,...,1.99548,0.306514,0.270139,0.268255,0.248476,0.059810,5.793295,0.115230,36.583354,20.47150
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149,...,0.52337,0.257826,0.241596,0.254830,0.089758,0.073190,0.359743,0.139530,-0.192774,25.33441


In [27]:
#Maximum left pupil diameter
max_lpd_data = []

for item in lpd_vector:
    file_key = item['file_key']
    lpd = item['LPD']

    #Check if the list is empty
    if lpd:  
        max_lpd = np.max(lpd)
    else:
        max_lpd = None

    max_lpd_data.append({
        'file_key': file_key,
        'f33': max_lpd
    })

# Create DataFrame 
max_lpd_df = pd.DataFrame(max_lpd_data)

feature_df = feature_df.merge(max_lpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f13,f14,f15,f16,f17,f18,f19,f20,f32,f33
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959,...,0.317581,0.264066,0.257935,0.350696,0.076720,6.272821,0.150022,41.389720,14.97845,21.82685
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898,...,0.349831,0.308199,0.294860,0.179607,0.114075,0.867020,0.266113,-0.160493,9.10561,23.06998
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440,...,0.265482,0.242950,0.244080,0.138565,0.044005,3.176352,0.086180,12.437481,14.20694,22.62979
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425,...,0.201254,0.156554,0.140630,0.287425,0.040160,7.426541,0.081420,60.650928,16.78424,22.31914
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911,...,0.281757,0.238197,0.241030,0.252794,0.063660,4.762304,0.123387,24.804236,16.44751,21.14130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665,...,0.396504,0.342790,0.311770,0.233169,0.097170,1.325127,0.241760,0.819504,25.92760,31.15751
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215,...,0.287688,0.261247,0.261295,0.138470,0.060455,1.884942,0.116243,5.161301,24.34467,32.63464
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985,...,0.306514,0.270139,0.268255,0.248476,0.059810,5.793295,0.115230,36.583354,20.47150,32.86868
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149,...,0.257826,0.241596,0.254830,0.089758,0.073190,0.359743,0.139530,-0.192774,25.33441,30.72208


In [28]:
#Arithmetic mean of left pupil diameter
mean_lpd_data = []

for item in lpd_vector:
    file_key = item['file_key']
    lpd = item['LPD']

    #Check if the list is empty
    if lpd:  
        mean_lpd = np.mean(lpd)
    else:
        mean_lpd = None

    mean_lpd_data.append({
        'file_key': file_key,
        'f34': mean_lpd
    })

# Create DataFrame 
mean_lpd_df = pd.DataFrame(mean_lpd_data)

feature_df = feature_df.merge(mean_lpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f14,f15,f16,f17,f18,f19,f20,f32,f33,f34
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959,...,0.264066,0.257935,0.350696,0.076720,6.272821,0.150022,41.389720,14.97845,21.82685,18.883407
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898,...,0.308199,0.294860,0.179607,0.114075,0.867020,0.266113,-0.160493,9.10561,23.06998,21.053556
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440,...,0.242950,0.244080,0.138565,0.044005,3.176352,0.086180,12.437481,14.20694,22.62979,19.978792
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425,...,0.156554,0.140630,0.287425,0.040160,7.426541,0.081420,60.650928,16.78424,22.31914,20.575229
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911,...,0.238197,0.241030,0.252794,0.063660,4.762304,0.123387,24.804236,16.44751,21.14130,18.973598
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665,...,0.342790,0.311770,0.233169,0.097170,1.325127,0.241760,0.819504,25.92760,31.15751,28.092948
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215,...,0.261247,0.261295,0.138470,0.060455,1.884942,0.116243,5.161301,24.34467,32.63464,28.787894
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985,...,0.270139,0.268255,0.248476,0.059810,5.793295,0.115230,36.583354,20.47150,32.86868,30.515380
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149,...,0.241596,0.254830,0.089758,0.073190,0.359743,0.139530,-0.192774,25.33441,30.72208,28.016040


In [29]:
#Geometric mean of left pupil diameter
gmean_lpd_data = []

for item in lpd_vector:
    file_key = item['file_key']
    lpd = item['LPD']

    #Check if the list is empty
    if lpd:  
        gmean_lpd = gmean(lpd)
    else:
        gmean_lpd = None

    gmean_lpd_data.append({
        'file_key': file_key,
        'f35': gmean_lpd
    })

# Create DataFrame 
gmean_lpd_df = pd.DataFrame(gmean_lpd_data)

feature_df = feature_df.merge(gmean_lpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f15,f16,f17,f18,f19,f20,f32,f33,f34,f35
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959,...,0.257935,0.350696,0.076720,6.272821,0.150022,41.389720,14.97845,21.82685,18.883407,18.793943
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898,...,0.294860,0.179607,0.114075,0.867020,0.266113,-0.160493,9.10561,23.06998,21.053556,21.035456
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440,...,0.244080,0.138565,0.044005,3.176352,0.086180,12.437481,14.20694,22.62979,19.978792,19.927327
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425,...,0.140630,0.287425,0.040160,7.426541,0.081420,60.650928,16.78424,22.31914,20.575229,20.561464
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911,...,0.241030,0.252794,0.063660,4.762304,0.123387,24.804236,16.44751,21.14130,18.973598,18.944220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665,...,0.311770,0.233169,0.097170,1.325127,0.241760,0.819504,25.92760,31.15751,28.092948,28.077206
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215,...,0.261295,0.138470,0.060455,1.884942,0.116243,5.161301,24.34467,32.63464,28.787894,28.732491
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985,...,0.268255,0.248476,0.059810,5.793295,0.115230,36.583354,20.47150,32.86868,30.515380,30.507258
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149,...,0.254830,0.089758,0.073190,0.359743,0.139530,-0.192774,25.33441,30.72208,28.016040,27.999061


In [30]:
#Median of left pupil diameter
median_lpd_data = []

for item in lpd_vector:
    file_key = item['file_key']
    lpd = item['LPD']

    #Check if the list is empty
    if lpd:  
        median_lpd = np.median(lpd)
    else:
        median_lpd = None

    median_lpd_data.append({
        'file_key': file_key,
        'f36': median_lpd
    })

# Create DataFrame 
median_lpd_df = pd.DataFrame(median_lpd_data)

feature_df = feature_df.merge(median_lpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f16,f17,f18,f19,f20,f32,f33,f34,f35,f36
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959,...,0.350696,0.076720,6.272821,0.150022,41.389720,14.97845,21.82685,18.883407,18.793943,18.824400
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898,...,0.179607,0.114075,0.867020,0.266113,-0.160493,9.10561,23.06998,21.053556,21.035456,21.097650
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440,...,0.138565,0.044005,3.176352,0.086180,12.437481,14.20694,22.62979,19.978792,19.927327,20.053690
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425,...,0.287425,0.040160,7.426541,0.081420,60.650928,16.78424,22.31914,20.575229,20.561464,20.493495
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911,...,0.252794,0.063660,4.762304,0.123387,24.804236,16.44751,21.14130,18.973598,18.944220,19.005680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665,...,0.233169,0.097170,1.325127,0.241760,0.819504,25.92760,31.15751,28.092948,28.077206,28.341340
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215,...,0.138470,0.060455,1.884942,0.116243,5.161301,24.34467,32.63464,28.787894,28.732491,28.803940
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985,...,0.248476,0.059810,5.793295,0.115230,36.583354,20.47150,32.86868,30.515380,30.507258,30.561960
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149,...,0.089758,0.073190,0.359743,0.139530,-0.192774,25.33441,30.72208,28.016040,27.999061,28.173940


In [31]:
#STD of left pupil diameter
std_lpd_data = []

for item in lpd_vector:
    file_key = item['file_key']
    lpd = item['LPD']

    #Check if the list is empty
    if lpd:  
        std_lpd = np.std(lpd)
    else:
        median_lpd = None

    std_lpd_data.append({
        'file_key': file_key,
        'f37': std_lpd
    })

# Create DataFrame 
std_lpd_df = pd.DataFrame(std_lpd_data)

feature_df = feature_df.merge(std_lpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f17,f18,f19,f20,f32,f33,f34,f35,f36,f37
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959,...,0.076720,6.272821,0.150022,41.389720,14.97845,21.82685,18.883407,18.793943,18.824400,1.821947
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898,...,0.114075,0.867020,0.266113,-0.160493,9.10561,23.06998,21.053556,21.035456,21.097650,0.835551
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440,...,0.044005,3.176352,0.086180,12.437481,14.20694,22.62979,19.978792,19.927327,20.053690,1.418854
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425,...,0.040160,7.426541,0.081420,60.650928,16.78424,22.31914,20.575229,20.561464,20.493495,0.750254
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911,...,0.063660,4.762304,0.123387,24.804236,16.44751,21.14130,18.973598,18.944220,19.005680,1.045301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665,...,0.097170,1.325127,0.241760,0.819504,25.92760,31.15751,28.092948,28.077206,28.341340,0.936039
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215,...,0.060455,1.884942,0.116243,5.161301,24.34467,32.63464,28.787894,28.732491,28.803940,1.771648
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985,...,0.059810,5.793295,0.115230,36.583354,20.47150,32.86868,30.515380,30.507258,30.561960,0.688562
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149,...,0.073190,0.359743,0.139530,-0.192774,25.33441,30.72208,28.016040,27.999061,28.173940,0.971654


In [32]:
#MAD of left pupil diameter
mad_lpd_data = []

for item in lpd_vector:
    file_key = item['file_key']
    lpd = item['LPD']

    #Check if the list is empty
    if lpd:  
        mad_lpd = median_abs_deviation(lpd, scale=1)
    else:
        median_lpd = None

    mad_lpd_data.append({
        'file_key': file_key,
        'f38': mad_lpd
    })

# Create DataFrame 
mad_lpd_df = pd.DataFrame(mad_lpd_data)

feature_df = feature_df.merge(mad_lpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f18,f19,f20,f32,f33,f34,f35,f36,f37,f38
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959,...,6.272821,0.150022,41.389720,14.97845,21.82685,18.883407,18.793943,18.824400,1.821947,1.773330
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898,...,0.867020,0.266113,-0.160493,9.10561,23.06998,21.053556,21.035456,21.097650,0.835551,0.373125
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440,...,3.176352,0.086180,12.437481,14.20694,22.62979,19.978792,19.927327,20.053690,1.418854,0.928220
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425,...,7.426541,0.081420,60.650928,16.78424,22.31914,20.575229,20.561464,20.493495,0.750254,0.577980
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911,...,4.762304,0.123387,24.804236,16.44751,21.14130,18.973598,18.944220,19.005680,1.045301,0.621870
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665,...,1.325127,0.241760,0.819504,25.92760,31.15751,28.092948,28.077206,28.341340,0.936039,0.574290
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215,...,1.884942,0.116243,5.161301,24.34467,32.63464,28.787894,28.732491,28.803940,1.771648,1.447990
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985,...,5.793295,0.115230,36.583354,20.47150,32.86868,30.515380,30.507258,30.561960,0.688562,0.389445
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149,...,0.359743,0.139530,-0.192774,25.33441,30.72208,28.016040,27.999061,28.173940,0.971654,0.703350


In [33]:
#SKEWNESS of left pupil diameter
skew_lpd_data = []

for item in lpd_vector:
    file_key = item['file_key']
    lpd = item['LPD']

    #Check if the list is empty
    if lpd:  
        skew_lpd = skew(lpd)
    else:
        median_lpd = None

    skew_lpd_data.append({
        'file_key': file_key,
        'f39': skew_lpd
    })

# Create DataFrame 
skew_lpd_df = pd.DataFrame(skew_lpd_data)

feature_df = feature_df.merge(skew_lpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f19,f20,f32,f33,f34,f35,f36,f37,f38,f39
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959,...,0.150022,41.389720,14.97845,21.82685,18.883407,18.793943,18.824400,1.821947,1.773330,-0.188199
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898,...,0.266113,-0.160493,9.10561,23.06998,21.053556,21.035456,21.097650,0.835551,0.373125,-2.127216
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440,...,0.086180,12.437481,14.20694,22.62979,19.978792,19.927327,20.053690,1.418854,0.928220,-0.341561
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425,...,0.081420,60.650928,16.78424,22.31914,20.575229,20.561464,20.493495,0.750254,0.577980,-0.187068
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911,...,0.123387,24.804236,16.44751,21.14130,18.973598,18.944220,19.005680,1.045301,0.621870,-0.460327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665,...,0.241760,0.819504,25.92760,31.15751,28.092948,28.077206,28.341340,0.936039,0.574290,-0.374585
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215,...,0.116243,5.161301,24.34467,32.63464,28.787894,28.732491,28.803940,1.771648,1.447990,-0.302612
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985,...,0.115230,36.583354,20.47150,32.86868,30.515380,30.507258,30.561960,0.688562,0.389445,-2.365517
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149,...,0.139530,-0.192774,25.33441,30.72208,28.016040,27.999061,28.173940,0.971654,0.703350,-0.278631


In [34]:
#IQR of left pupil diameter
iqr_lpd_data = []

for item in lpd_vector:
    file_key = item['file_key']
    lpd = item['LPD']

    #Check if the list is empty
    if lpd:  
        iqr_lpd = iqr(lpd)
    else:
        median_lpd = None

    iqr_lpd_data.append({
        'file_key': file_key,
        'f40': iqr_lpd
    })

# Create DataFrame 
iqr_lpd_df = pd.DataFrame(iqr_lpd_data)

feature_df = feature_df.merge(iqr_lpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f20,f32,f33,f34,f35,f36,f37,f38,f39,f40
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959,...,41.389720,14.97845,21.82685,18.883407,18.793943,18.824400,1.821947,1.773330,-0.188199,3.507080
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898,...,-0.160493,9.10561,23.06998,21.053556,21.035456,21.097650,0.835551,0.373125,-2.127216,0.751947
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440,...,12.437481,14.20694,22.62979,19.978792,19.927327,20.053690,1.418854,0.928220,-0.341561,1.861880
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425,...,60.650928,16.78424,22.31914,20.575229,20.561464,20.493495,0.750254,0.577980,-0.187068,1.184795
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911,...,24.804236,16.44751,21.14130,18.973598,18.944220,19.005680,1.045301,0.621870,-0.460327,1.245265
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665,...,0.819504,25.92760,31.15751,28.092948,28.077206,28.341340,0.936039,0.574290,-0.374585,1.317890
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215,...,5.161301,24.34467,32.63464,28.787894,28.732491,28.803940,1.771648,1.447990,-0.302612,2.953250
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985,...,36.583354,20.47150,32.86868,30.515380,30.507258,30.561960,0.688562,0.389445,-2.365517,0.778917
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149,...,-0.192774,25.33441,30.72208,28.016040,27.999061,28.173940,0.971654,0.703350,-0.278631,1.494210


In [35]:
#Kurtosis of left pupil diameter
kurt_lpd_data = []

for item in lpd_vector:
    file_key = item['file_key']
    lpd = item['LPD']

    #Check if the list is empty
    if lpd:  
        kurt_lpd = kurtosis(lpd)
    else:
        median_lpd = None

    kurt_lpd_data.append({
        'file_key': file_key,
        'f41': kurt_lpd
    })

# Create DataFrame 
kurt_lpd_df = pd.DataFrame(kurt_lpd_data)

feature_df = feature_df.merge(kurt_lpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959,...,14.97845,21.82685,18.883407,18.793943,18.824400,1.821947,1.773330,-0.188199,3.507080,-1.330764
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898,...,9.10561,23.06998,21.053556,21.035456,21.097650,0.835551,0.373125,-2.127216,0.751947,23.820888
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440,...,14.20694,22.62979,19.978792,19.927327,20.053690,1.418854,0.928220,-0.341561,1.861880,-0.529504
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425,...,16.78424,22.31914,20.575229,20.561464,20.493495,0.750254,0.577980,-0.187068,1.184795,-0.021650
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911,...,16.44751,21.14130,18.973598,18.944220,19.005680,1.045301,0.621870,-0.460327,1.245265,-0.385668
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665,...,25.92760,31.15751,28.092948,28.077206,28.341340,0.936039,0.574290,-0.374585,1.317890,-0.399668
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215,...,24.34467,32.63464,28.787894,28.732491,28.803940,1.771648,1.447990,-0.302612,2.953250,-0.581471
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985,...,20.47150,32.86868,30.515380,30.507258,30.561960,0.688562,0.389445,-2.365517,0.778917,27.484172
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149,...,25.33441,30.72208,28.016040,27.999061,28.173940,0.971654,0.703350,-0.278631,1.494210,-0.477196


In [36]:
#List of all valid(RPV = 1) right pupil diameter(RPD) for each file
rpd_vector = []

# Loop through all DataFrames in the dictionary
for key, df in all_dfs.items():
    # Query using DuckDB
    result = duckdb.query("""
        SELECT RPD 
        FROM df
        WHERE RPV = '1'
    """).to_df()

    # Extract vectors
    rpd_list = result['RPD'].tolist()

    # Append result 
    rpd_vector.append({
        'file_key': key,
        'RPD': rpd_list
    })

# Create the DataFrame
rpd_vector_df = pd.DataFrame(rpd_vector)

# Display the DataFrame
display(rpd_vector_df)


,file_key,RPD
0,T1_S1_TRY1_VB_SL_LIT,"[17.2877, 17.82784, 18.85434, 18.85434, 18.854..."
1,T1_S1_TRY1_HS_SL_LIT,"[19.8148, 19.63718, 19.91414, 19.88341, 20.081..."
2,T1_S1_TRY1_VB_SL_BIG,"[16.83182, 17.0567, 16.94397, 16.91181, 16.819..."
3,T1_S1_TRY1_HS_SL_BIG,"[16.64259, 18.41689, 18.45479, 18.44613, 18.48..."
4,T1_S1_TRY1_VB_FA_LIT,"[19.08659, 18.89192, 18.82117, 18.85699, 18.70..."
...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,"[29.19351, 29.10667, 28.34218, 28.2825, 28.261..."
2324,T33_S3_TRY3_VB_FA_LIT,"[28.86482, 27.62969, 27.63639, 27.63096, 27.64..."
2325,T33_S3_TRY3_HS_FA_LIT,"[26.51647, 26.5827, 29.29068, 29.39616, 29.267..."
2326,T33_S3_TRY3_VB_FA_BIG,"[27.11602, 27.07738, 27.06608, 27.06064, 27.07..."


In [37]:
#Minimum right pupil diameter
min_rpd_data = []

for item in rpd_vector:
    file_key = item['file_key']
    rpd = item['RPD']

    #Check if the list is empty
    if rpd:  
        min_rpd = np.min(rpd)
    else:
        min_rpd = None

    min_rpd_data.append({
        'file_key': file_key,
        'f42': min_rpd
    })

# Create DataFrame from min differences
min_rpd_df = pd.DataFrame(min_rpd_data)

feature_df = feature_df.merge(min_rpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959,...,21.82685,18.883407,18.793943,18.824400,1.821947,1.773330,-0.188199,3.507080,-1.330764,11.73673
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898,...,23.06998,21.053556,21.035456,21.097650,0.835551,0.373125,-2.127216,0.751947,23.820888,11.33286
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440,...,22.62979,19.978792,19.927327,20.053690,1.418854,0.928220,-0.341561,1.861880,-0.529504,10.67522
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425,...,22.31914,20.575229,20.561464,20.493495,0.750254,0.577980,-0.187068,1.184795,-0.021650,12.66637
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911,...,21.14130,18.973598,18.944220,19.005680,1.045301,0.621870,-0.460327,1.245265,-0.385668,16.05203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665,...,31.15751,28.092948,28.077206,28.341340,0.936039,0.574290,-0.374585,1.317890,-0.399668,23.58247
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215,...,32.63464,28.787894,28.732491,28.803940,1.771648,1.447990,-0.302612,2.953250,-0.581471,22.72544
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985,...,32.86868,30.515380,30.507258,30.561960,0.688562,0.389445,-2.365517,0.778917,27.484172,15.74079
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149,...,30.72208,28.016040,27.999061,28.173940,0.971654,0.703350,-0.278631,1.494210,-0.477196,22.30473


In [38]:
#Maximum right pupil diameter
max_rpd_data = []

for item in rpd_vector:
    file_key = item['file_key']
    rpd = item['RPD']

    #Check if the list is empty
    if rpd:  
        max_rpd = np.max(rpd)
    else:
        max_rpd = None

    max_rpd_data.append({
        'file_key': file_key,
        'f43': max_rpd
    })

# Create DataFrame 
max_rpd_df = pd.DataFrame(max_rpd_data)

feature_df = feature_df.merge(max_rpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)


,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959,...,18.883407,18.793943,18.824400,1.821947,1.773330,-0.188199,3.507080,-1.330764,11.73673,20.96601
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898,...,21.053556,21.035456,21.097650,0.835551,0.373125,-2.127216,0.751947,23.820888,11.33286,21.62618
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440,...,19.978792,19.927327,20.053690,1.418854,0.928220,-0.341561,1.861880,-0.529504,10.67522,21.29832
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425,...,20.575229,20.561464,20.493495,0.750254,0.577980,-0.187068,1.184795,-0.021650,12.66637,21.50403
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911,...,18.973598,18.944220,19.005680,1.045301,0.621870,-0.460327,1.245265,-0.385668,16.05203,20.68424
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665,...,28.092948,28.077206,28.341340,0.936039,0.574290,-0.374585,1.317890,-0.399668,23.58247,29.19351
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215,...,28.787894,28.732491,28.803940,1.771648,1.447990,-0.302612,2.953250,-0.581471,22.72544,29.06525
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985,...,30.515380,30.507258,30.561960,0.688562,0.389445,-2.365517,0.778917,27.484172,15.74079,30.78447
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149,...,28.016040,27.999061,28.173940,0.971654,0.703350,-0.278631,1.494210,-0.477196,22.30473,28.92718


In [39]:
#Arithmetic mean of right pupil diameter
mean_rpd_data = []

for item in rpd_vector:
    file_key = item['file_key']
    rpd = item['RPD']

    #Check if the list is empty
    if rpd:  
        mean_rpd = np.mean(rpd)
    else:
        mean_rpd = None

    mean_rpd_data.append({
        'file_key': file_key,
        'f44': mean_rpd
    })

# Create DataFrame 
mean_rpd_df = pd.DataFrame(mean_rpd_data)

feature_df = feature_df.merge(mean_rpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959,...,18.793943,18.824400,1.821947,1.773330,-0.188199,3.507080,-1.330764,11.73673,20.96601,18.064635
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898,...,21.035456,21.097650,0.835551,0.373125,-2.127216,0.751947,23.820888,11.33286,21.62618,19.355156
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440,...,19.927327,20.053690,1.418854,0.928220,-0.341561,1.861880,-0.529504,10.67522,21.29832,18.887727
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425,...,20.561464,20.493495,0.750254,0.577980,-0.187068,1.184795,-0.021650,12.66637,21.50403,18.846288
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911,...,18.944220,19.005680,1.045301,0.621870,-0.460327,1.245265,-0.385668,16.05203,20.68424,18.653069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665,...,28.077206,28.341340,0.936039,0.574290,-0.374585,1.317890,-0.399668,23.58247,29.19351,26.197818
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215,...,28.732491,28.803940,1.771648,1.447990,-0.302612,2.953250,-0.581471,22.72544,29.06525,26.071566
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985,...,30.507258,30.561960,0.688562,0.389445,-2.365517,0.778917,27.484172,15.74079,30.78447,28.055571
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149,...,27.999061,28.173940,0.971654,0.703350,-0.278631,1.494210,-0.477196,22.30473,28.92718,25.675444


In [40]:
#Geometric mean of right pupil diameter
gmean_rpd_data = []

for item in rpd_vector:
    file_key = item['file_key']
    rpd = item['RPD']

    #Check if the list is empty
    if rpd:  
        gmean_rpd = gmean(rpd)
    else:
        gmean_rpd = None

    gmean_rpd_data.append({
        'file_key': file_key,
        'f45': gmean_rpd
    })

# Create DataFrame 
gmean_rpd_df = pd.DataFrame(gmean_rpd_data)

feature_df = feature_df.merge(gmean_rpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959,...,18.824400,1.821947,1.773330,-0.188199,3.507080,-1.330764,11.73673,20.96601,18.064635,17.999585
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898,...,21.097650,0.835551,0.373125,-2.127216,0.751947,23.820888,11.33286,21.62618,19.355156,19.328542
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440,...,20.053690,1.418854,0.928220,-0.341561,1.861880,-0.529504,10.67522,21.29832,18.887727,18.844825
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425,...,20.493495,0.750254,0.577980,-0.187068,1.184795,-0.021650,12.66637,21.50403,18.846288,18.807152
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911,...,19.005680,1.045301,0.621870,-0.460327,1.245265,-0.385668,16.05203,20.68424,18.653069,18.622358
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665,...,28.341340,0.936039,0.574290,-0.374585,1.317890,-0.399668,23.58247,29.19351,26.197818,26.187200
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215,...,28.803940,1.771648,1.447990,-0.302612,2.953250,-0.581471,22.72544,29.06525,26.071566,26.021556
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985,...,30.561960,0.688562,0.389445,-2.365517,0.778917,27.484172,15.74079,30.78447,28.055571,28.044514
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149,...,28.173940,0.971654,0.703350,-0.278631,1.494210,-0.477196,22.30473,28.92718,25.675444,25.645190


In [41]:
#Median of right pupil diameter
median_rpd_data = []

for item in rpd_vector:
    file_key = item['file_key']
    rpd = item['RPD']

    #Check if the list is empty
    if rpd:  
        median_rpd = np.median(rpd)
    else:
        median_rpd = None

    median_rpd_data.append({
        'file_key': file_key,
        'f46': median_rpd
    })

# Create DataFrame 
median_rpd_df = pd.DataFrame(median_rpd_data)

feature_df = feature_df.merge(median_rpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959,...,1.821947,1.773330,-0.188199,3.507080,-1.330764,11.73673,20.96601,18.064635,17.999585,18.060460
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898,...,0.835551,0.373125,-2.127216,0.751947,23.820888,11.33286,21.62618,19.355156,19.328542,19.203055
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440,...,1.418854,0.928220,-0.341561,1.861880,-0.529504,10.67522,21.29832,18.887727,18.844825,19.103850
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425,...,0.750254,0.577980,-0.187068,1.184795,-0.021650,12.66637,21.50403,18.846288,18.807152,18.396135
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911,...,1.045301,0.621870,-0.460327,1.245265,-0.385668,16.05203,20.68424,18.653069,18.622358,18.833000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665,...,0.936039,0.574290,-0.374585,1.317890,-0.399668,23.58247,29.19351,26.197818,26.187200,26.345650
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215,...,1.771648,1.447990,-0.302612,2.953250,-0.581471,22.72544,29.06525,26.071566,26.021556,26.163730
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985,...,0.688562,0.389445,-2.365517,0.778917,27.484172,15.74079,30.78447,28.055571,28.044514,28.043600
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149,...,0.971654,0.703350,-0.278631,1.494210,-0.477196,22.30473,28.92718,25.675444,25.645190,25.683990


In [42]:
#STD of right pupil diameter
std_rpd_data = []

for item in rpd_vector:
    file_key = item['file_key']
    rpd = item['RPD']

    #Check if the list is empty
    if rpd:  
        std_rpd = np.std(rpd)
    else:
        median_rpd = None

    std_rpd_data.append({
        'file_key': file_key,
        'f47': std_rpd
    })

# Create DataFrame 
std_rpd_df = pd.DataFrame(std_rpd_data)

feature_df = feature_df.merge(std_rpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959,...,1.773330,-0.188199,3.507080,-1.330764,11.73673,20.96601,18.064635,17.999585,18.060460,1.527503
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898,...,0.373125,-2.127216,0.751947,23.820888,11.33286,21.62618,19.355156,19.328542,19.203055,1.005368
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440,...,0.928220,-0.341561,1.861880,-0.529504,10.67522,21.29832,18.887727,18.844825,19.103850,1.258265
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425,...,0.577980,-0.187068,1.184795,-0.021650,12.66637,21.50403,18.846288,18.807152,18.396135,1.226277
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911,...,0.621870,-0.460327,1.245265,-0.385668,16.05203,20.68424,18.653069,18.622358,18.833000,1.058681
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665,...,0.574290,-0.374585,1.317890,-0.399668,23.58247,29.19351,26.197818,26.187200,26.345650,0.741422
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215,...,1.447990,-0.302612,2.953250,-0.581471,22.72544,29.06525,26.071566,26.021556,26.163730,1.611523
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985,...,0.389445,-2.365517,0.778917,27.484172,15.74079,30.78447,28.055571,28.044514,28.043600,0.755608
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149,...,0.703350,-0.278631,1.494210,-0.477196,22.30473,28.92718,25.675444,25.645190,25.683990,1.243970


In [43]:
#MAD of right pupil diameter
mad_rpd_data = []

for item in rpd_vector:
    file_key = item['file_key']
    rpd = item['RPD']

    #Check if the list is empty
    if rpd:  
        mad_rpd = median_abs_deviation(rpd, scale=1)
    else:
        median_rpd = None

    mad_rpd_data.append({
        'file_key': file_key,
        'f48': mad_rpd
    })

# Create DataFrame 
mad_rpd_df = pd.DataFrame(mad_rpd_data)

feature_df = feature_df.merge(mad_rpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959,...,-0.188199,3.507080,-1.330764,11.73673,20.96601,18.064635,17.999585,18.060460,1.527503,1.443570
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898,...,-2.127216,0.751947,23.820888,11.33286,21.62618,19.355156,19.328542,19.203055,1.005368,0.669080
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440,...,-0.341561,1.861880,-0.529504,10.67522,21.29832,18.887727,18.844825,19.103850,1.258265,0.864570
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425,...,-0.187068,1.184795,-0.021650,12.66637,21.50403,18.846288,18.807152,18.396135,1.226277,0.647955
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911,...,-0.460327,1.245265,-0.385668,16.05203,20.68424,18.653069,18.622358,18.833000,1.058681,0.690400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665,...,-0.374585,1.317890,-0.399668,23.58247,29.19351,26.197818,26.187200,26.345650,0.741422,0.389580
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215,...,-0.302612,2.953250,-0.581471,22.72544,29.06525,26.071566,26.021556,26.163730,1.611523,1.450710
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985,...,-2.365517,0.778917,27.484172,15.74079,30.78447,28.055571,28.044514,28.043600,0.755608,0.413755
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149,...,-0.278631,1.494210,-0.477196,22.30473,28.92718,25.675444,25.645190,25.683990,1.243970,0.713330


In [44]:
#SKEWNESS of right pupil diameter
skew_rpd_data = []

for item in rpd_vector:
    file_key = item['file_key']
    rpd = item['RPD']

    #Check if the list is empty
    if rpd:  
        skew_rpd = skew(rpd)
    else:
        median_rpd = None

    skew_rpd_data.append({
        'file_key': file_key,
        'f49': skew_rpd
    })

# Create DataFrame 
skew_rpd_df = pd.DataFrame(skew_rpd_data)

feature_df = feature_df.merge(skew_rpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959,...,3.507080,-1.330764,11.73673,20.96601,18.064635,17.999585,18.060460,1.527503,1.443570,-0.037453
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898,...,0.751947,23.820888,11.33286,21.62618,19.355156,19.328542,19.203055,1.005368,0.669080,-0.295182
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440,...,1.861880,-0.529504,10.67522,21.29832,18.887727,18.844825,19.103850,1.258265,0.864570,-0.398767
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425,...,1.184795,-0.021650,12.66637,21.50403,18.846288,18.807152,18.396135,1.226277,0.647955,0.535961
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911,...,1.245265,-0.385668,16.05203,20.68424,18.653069,18.622358,18.833000,1.058681,0.690400,-0.505731
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665,...,1.317890,-0.399668,23.58247,29.19351,26.197818,26.187200,26.345650,0.741422,0.389580,-0.560146
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215,...,2.953250,-0.581471,22.72544,29.06525,26.071566,26.021556,26.163730,1.611523,1.450710,-0.037291
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985,...,0.778917,27.484172,15.74079,30.78447,28.055571,28.044514,28.043600,0.755608,0.413755,-3.319845
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149,...,1.494210,-0.477196,22.30473,28.92718,25.675444,25.645190,25.683990,1.243970,0.713330,-0.035704


In [45]:
#IQR of right pupil diameter
iqr_rpd_data = []

for item in rpd_vector:
    file_key = item['file_key']
    rpd = item['RPD']

    #Check if the list is empty
    if rpd:  
        iqr_rpd = iqr(rpd)
    else:
        median_rpd = None

    iqr_rpd_data.append({
        'file_key': file_key,
        'f50': iqr_rpd
    })

# Create DataFrame 
iqr_rpd_df = pd.DataFrame(iqr_rpd_data)

feature_df = feature_df.merge(iqr_rpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959,...,-1.330764,11.73673,20.96601,18.064635,17.999585,18.060460,1.527503,1.443570,-0.037453,2.875732
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898,...,23.820888,11.33286,21.62618,19.355156,19.328542,19.203055,1.005368,0.669080,-0.295182,1.408065
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440,...,-0.529504,10.67522,21.29832,18.887727,18.844825,19.103850,1.258265,0.864570,-0.398767,2.183140
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425,...,-0.021650,12.66637,21.50403,18.846288,18.807152,18.396135,1.226277,0.647955,0.535961,2.123840
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911,...,-0.385668,16.05203,20.68424,18.653069,18.622358,18.833000,1.058681,0.690400,-0.505731,1.534745
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665,...,-0.399668,23.58247,29.19351,26.197818,26.187200,26.345650,0.741422,0.389580,-0.560146,0.843050
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215,...,-0.581471,22.72544,29.06525,26.071566,26.021556,26.163730,1.611523,1.450710,-0.037291,2.886730
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985,...,27.484172,15.74079,30.78447,28.055571,28.044514,28.043600,0.755608,0.413755,-3.319845,0.815325
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149,...,-0.477196,22.30473,28.92718,25.675444,25.645190,25.683990,1.243970,0.713330,-0.035704,1.403100


In [46]:
#Kurtosis of right pupil diameter
kurt_rpd_data = []

for item in rpd_vector:
    file_key = item['file_key']
    rpd = item['RPD']

    #Check if the list is empty
    if rpd:  
        kurt_rpd = kurtosis(rpd)
    else:
        median_rpd = None

    kurt_rpd_data.append({
        'file_key': file_key,
        'f51': kurt_rpd
    })

# Create DataFrame 
kurt_rpd_df = pd.DataFrame(kurt_rpd_data)

feature_df = feature_df.merge(kurt_rpd_df, on='file_key', how='left')
display(feature_df)

#Overwrite the csv 
feature_df.to_csv(r'C:\Users\david\OneDrive\Documenti\Tesi_BehavBio\Programs\Feature_csv\feature_vector.csv', index=False)

,file_key,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51
0,T1_S1_TRY1_VB_SL_LIT,57,0.04688,0.60925,0.251515,0.228373,0.246580,0.108956,0.071780,0.900959,...,11.73673,20.96601,18.064635,17.999585,18.060460,1.527503,1.443570,-0.037453,2.875732,-1.121120
1,T1_S1_TRY1_HS_SL_LIT,45,0.11426,0.77112,0.322219,0.282340,0.267210,0.172174,0.099730,0.966898,...,11.33286,21.62618,19.355156,19.328542,19.203055,1.005368,0.669080,-0.295182,1.408065,2.610415
2,T1_S1_TRY1_VB_SL_BIG,59,0.03345,0.93579,0.239702,0.215981,0.226560,0.128417,0.041380,3.328440,...,10.67522,21.29832,18.887727,18.844825,19.103850,1.258265,0.864570,-0.398767,2.183140,-0.324120
3,T1_S1_TRY1_HS_SL_BIG,90,0.01978,0.72949,0.154106,0.133018,0.120850,0.106278,0.032900,3.101425,...,12.66637,21.50403,18.846288,18.807152,18.396135,1.226277,0.647955,0.535961,2.123840,-0.584666
4,T1_S1_TRY1_VB_FA_LIT,65,0.02087,1.28003,0.238144,0.205979,0.221190,0.159147,0.065310,4.322911,...,16.05203,20.68424,18.653069,18.622358,18.833000,1.058681,0.690400,-0.505731,1.534745,-0.680287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,T33_S3_TRY3_HS_SL_BIG,42,0.09625,0.99805,0.373094,0.319370,0.300230,0.225034,0.104915,1.419665,...,23.58247,29.19351,26.197818,26.187200,26.345650,0.741422,0.389580,-0.560146,0.843050,1.090687
2324,T33_S3_TRY3_VB_FA_LIT,57,0.02625,0.62970,0.261618,0.234139,0.247500,0.116631,0.061700,0.879215,...,22.72544,29.06525,26.071566,26.021556,26.163730,1.611523,1.450710,-0.037291,2.886730,-1.170000
2325,T33_S3_TRY3_HS_FA_LIT,55,0.06659,1.94910,0.291847,0.253836,0.261050,0.242696,0.059450,5.758985,...,15.74079,30.78447,28.055571,28.044514,28.043600,0.755608,0.413755,-3.319845,0.815325,47.851231
2326,T33_S3_TRY3_VB_FA_BIG,58,0.09448,0.51691,0.247106,0.227703,0.245025,0.094850,0.080200,0.305149,...,22.30473,28.92718,25.675444,25.645190,25.683990,1.243970,0.713330,-0.035704,1.403100,-0.159372
